# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import pandas as pd
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)
%matplotlib inline
# Load/Prep the Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape Before:', x_train.shape)
print('y_train shape Before:', y_train.reshape(50000).shape, )


x_train_lt5 = x_train[y_train.reshape(50000) < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test.reshape(10000) < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train.reshape(50000) >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test.reshape(10000) >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step
x_train shape Before: (50000, 32, 32, 3)
y_train shape Before: (50000,)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5 = np_utils.to_categorical(y_train_lt5, 5)
y_test_lt5 = np_utils.to_categorical(y_test_lt5, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [3]:
batch_size = 32
num_classes = 5
epochs = 100
model = Sequential()
model.add(Convolution2D(32, (3, 3), padding='same', input_shape=x_train_lt5.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')
x_train_lt5 /= 255
x_test_lt5 /= 255

model.fit(x_train_lt5, y_train_lt5, batch_size=batch_size, epochs=epochs, validation_data=(x_test_lt5, y_test_lt5), shuffle=True)

W0623 06:00:23.900361 139656415602560 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 06:00:23.947627 139656415602560 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 06:00:23.954618 139656415602560 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 06:00:23.994750 139656415602560 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0623 06:00:23.997526 139656415602560 deprecation_wrapp

Train on 25000 samples, validate on 5000 samples
Epoch 1/100
25000/25000 [==============================] - 12s 491us/step - loss: 1.2720 - acc: 0.4629 - val_loss: 1.1524 - val_acc: 0.5386
Epoch 2/100
25000/25000 [==============================] - 5s 220us/step - loss: 1.0364 - acc: 0.5792 - val_loss: 0.9450 - val_acc: 0.6374
Epoch 3/100
25000/25000 [==============================] - 5s 216us/step - loss: 0.9436 - acc: 0.6196 - val_loss: 0.9028 - val_acc: 0.6536
Epoch 4/100
25000/25000 [==============================] - 5s 212us/step - loss: 0.8830 - acc: 0.6491 - val_loss: 0.8479 - val_acc: 0.6592
Epoch 5/100
25000/25000 [==============================] - 5s 212us/step - loss: 0.8407 - acc: 0.6686 - val_loss: 0.7847 - val_acc: 0.6912
Epoch 6/100
25000/25000 [==============================] - 5s 212us/step - loss: 0.8023 - acc: 0.6864 - val_loss: 0.7608 - val_acc: 0.7064
Epoch 7/100
25000/25000 [==============================] - 5s 211us/step - loss: 0.7723 - acc: 0.7026 - val_loss: 0.

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [5]:
for layer in model.layers[:11]:
    layer.trainable=False
for layer in model.layers[11:]:
    layer.trainable=True
    
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d_1
False
activation_1
False
conv2d_2
False
activation_2
False
max_pooling2d_1
False
dropout_1
False
conv2d_3
False
activation_3
False
conv2d_4
False
activation_4
False
max_pooling2d_2
False
dropout_2
True
flatten_1
True
dense_1
True
activation_5
True
dropout_3
True
dense_2
True
activation_6
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [6]:
x_train_gte5 = x_train_gte5.astype('float32')
x_test_gte5 = x_test_gte5.astype('float32')
x_train_gte5 /= 255
x_test_gte5 /= 255
y_train_gte5 = np_utils.to_categorical(y_train_gte5, 5)
y_test_gte5 = np_utils.to_categorical(y_test_gte5, 5)
model.fit(x_train_gte5, y_train_gte5, batch_size=batch_size, nb_epoch=epochs, validation_data=(x_test_gte5, y_test_gte5))

Train on 25000 samples, validate on 5000 samples
Epoch 1/100
  608/25000 [..............................] - ETA: 6s - loss: 2.2182 - acc: 0.1645

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 5s 219us/step - loss: 1.1429 - acc: 0.5360 - val_loss: 0.8154 - val_acc: 0.6960
Epoch 2/100
25000/25000 [==============================] - 5s 210us/step - loss: 0.7409 - acc: 0.7292 - val_loss: 0.6853 - val_acc: 0.7396
Epoch 3/100
25000/25000 [==============================] - 5s 209us/step - loss: 0.6406 - acc: 0.7690 - val_loss: 0.6131 - val_acc: 0.7856
Epoch 4/100
25000/25000 [==============================] - 5s 210us/step - loss: 0.5770 - acc: 0.7932 - val_loss: 0.5160 - val_acc: 0.8074
Epoch 5/100
25000/25000 [==============================] - 5s 210us/step - loss: 0.5430 - acc: 0.8079 - val_loss: 0.6074 - val_acc: 0.7726
Epoch 6/100
25000/25000 [==============================] - 5s 211us/step - loss: 0.5057 - acc: 0.8215 - val_loss: 0.4231 - val_acc: 0.8512
Epoch 7/100
25000/25000 [==============================] - 5s 211us/step - loss: 0.4845 - acc: 0.8306 - val_loss: 0.4406 - val_acc: 0.8426
Epoch 8/100
25000/25000 [==============

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [18]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [19]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [20]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [23]:
type(twenty_train)

sklearn.utils.Bunch

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer(stop_words='english')
dtm = vect.fit_transform(twenty_train.data)
dtm_test = vect.transform(twenty_test.data)
features = vect.get_feature_names()
print(dtm.shape,dtm_test.shape)

(2257, 35482) (1502, 35482)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression(C=1e9)
logreg.fit(dtm, twenty_train.target)
y_pred_class = logreg.predict(dtm_test)
print(metrics.accuracy_score(twenty_train.target, logreg.predict(dtm)))
print(metrics.accuracy_score(twenty_test.target, y_pred_class))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


1.0
0.9280958721704394
